CODE TEST QDRANT
- Connect Qdrant server (thay localhost bằng IP máy chủ)
- Tạo locations
- Embeding pdf file (change pdf_path)

Note: befor running code make sure to
- Create and active environment
- install Qdrant client:  pip install qdrant-client[fastembed]

FastEmbed is a library for creating fast vector embeddings on CPU. It is based on ONNX Runtime and allows to run inference on CPU with GPU-like performance.
Qdrant Client can use FastEmbed to create embeddings and upload them to Qdrant. This allows to simplify API and make it more intuitive.

In [31]:
# pip install qdrant-client

In [18]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import os
import pdfplumber
from sentence_transformers import SentenceTransformer

# Initialize Qdrant client
qdrant_client = QdrantClient("localhost:6333")  # Replace with your Qdrant server URL

# Initialize the BERT model for sentence embedding
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [23]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = []
        for i, page in enumerate(pdf.pages):
            text.append(page.extract_text())
    return text

In [25]:
# Path to the PDF file you want to embed
pdf_path = '/Users/nad/Desktop/test_pdf/Microsoft_Press_ebook_Introducing_Power_BI_PDF_mobile.pdf'
# Define the collection name as "First_collection"
collection_name = "power_bi_fundamentals_remote_test"
# Check if the collection exists, if not create it
collections = qdrant_client.get_collections().collections
collection_names = [collection.name for collection in collections]
if collection_name not in collection_names:
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(size=384, distance=models.Distance.COSINE),
        
    )
    print(f"Collection '{collection_name}' created.")

In [26]:
# Function to convert text to embeddings and save to Qdrant for a single PDF
text = extract_text_from_pdf(pdf_path)

In [27]:
points = []
for i in range(len(text)):
    embeded_vector = model.encode(text[i]).tolist()
    point = models.PointStruct(
                id=i,
                payload={
                    "text": text[i],
                },
                vector=embeded_vector,
            )
    points.append(point)

qdrant_client.upsert(
    collection_name=collection_name,
    points=points)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)